In [63]:
using Test

In [64]:
f = readlines("day16.input");

In [65]:
function parse_input(input)
    flows = Dict{String, Int64}()
    edges = Dict{String, Vector{String}}()
    for line in input
        name_re = r"Valve ([A-Z][A-Z]) has flow rate=\d+; tunnels? leads? to valves? .*"
        flow_re = r"Valve [A-Z][A-Z] has flow rate=(\d+); tunnels? leads? to valves? .*"
        edge_re = r"Valve [A-Z][A-Z] has flow rate=\d+; tunnels? leads? to valves? (.*)"
        name = first(match(name_re, line).captures)
        flows[name] = parse(Int64, first(match(flow_re, line).captures))
        edges[name] = [strip(x) for x in (split(first(match(edge_re, line).captures), ","))]
    end
    flows, edges
end

parse_input (generic function with 1 method)

In [66]:
function create_transform(locations)
    transform = Dict()
    alphabetical = sort(collect(locations))
    for (i, key) in enumerate(alphabetical)
        transform[key] = i
    end

    transform
end

create_transform (generic function with 1 method)

In [67]:
function step(pos, time, open, adjacent, flows, states)
    score = states[(pos, time, open)]

    for adj in adjacent
        previous_max = get(states, (adj, time+1, open), -1)
        states[(adj, time+1, open)] = max(previous_max, score)
    end    
    if flows[pos] > 0 && open & 1<<pos == 0
        previous_max = get(states, (pos, time+1, open | 1<<pos), -1)
        states[(pos, time+1, open | 1<<pos)] = max(previous_max, score + flows[pos]*(30-time))
    end
end

step (generic function with 1 method)

In [68]:
function solve_part_1(input)
    alphabet_flows, alphabet_edges = parse_input(input)
    transform = create_transform(keys(alphabet_flows))
    flows = Dict{Int64, Int64}()
    edges = Dict{Int64, Vector{Int64}}()
    for (k, v) in alphabet_flows
        flows[transform[k]] = v
    end
    for (k, v) in alphabet_edges
        edges[transform[k]] = [transform[x] for x in v]
    end

    states = Dict()
    states[(1, 1, 0)] = 0

    for i=1:30
        for state in keys(states)
            if state[2] == i
                step(state[1], i, state[3], edges[state[1]], flows, states)
            end
        end

        # perf: remove poorly performing states based on arbitrary heuristic... if answer is wrong, cull fewer states here
        if i%9 == 0
            best = maximum(values(states))
            states = filter!(x -> x[2]>0.9*best, states)
        end
    end

    println("Stored state for $(length(states)) problems")
    maximum(values(states))
end

solve_part_1 (generic function with 1 method)

In [69]:
@test solve_part_1(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II", "\n"))) == 1651


Stored state for 156 problems


Test Passed

In [70]:
solve_part_1(f)

Stored state for 2434 problems


2056